In [1]:
from pyspark.sql import SparkSession

# Spark session and context
spark = (
    SparkSession
    .builder
    .master("local")
    .appName("kafka-streaming")
    # Add kafka package
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
    .getOrCreate()
)

sc = spark.sparkContext

In [4]:
# Read the message from Kafka stream
df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "ingestion-topic") \
    .load()
df.selectExpr("CAST (key AS STRING)", "CAST (value AS STRING)")

DataFrame[key: string, value: string]

In [5]:
# Create a temporary view of SparkSQL
df.createOrReplaceTempView("message")

In [6]:
# Write down the message to the console of environment
res = spark.sql("SELECT * FROM message")
res.writeStream.format("console") \
            .outputMode("append") \
            .start()

In [ ]:
# Write the message back into Kafka in another stream
ds = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "spark-output") \
    .option("checkpointLocation", "/tmp") \
    .start() \
    .awaitTermination()